# STATISTICAL DATA ANALYSIS and MACHINE LEARNING 

You will find descriptions about the cell almost above each cell. I will make the general explanation here. Our project is based on statistical analysis of 6 different machine learning algorithms over 3 databases. Although the structure of our databases varies, we will examine how different algorithms work on different sets of the same algorithms or on the same sets. The code block you see below is the experimental stage of this thesis.

### 1. Loading the libraries

At the beginning of our project, we need to add the libraries required for our project. You can see all the libraries added in the cell below. 

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from PIL import Image
from sklearn.metrics import accuracy_score

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM

from matplotlib import pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.feature_selection import RFE

### 2. Loading the models

We load 6 blank machine learning models after adding our libraries. 

In [2]:
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs = -1)
logreg = LogisticRegression(solver='liblinear',multi_class='ovr')
svm = SVC(gamma='auto')
xgb = xgb.XGBClassifier()
gnb = GaussianNB()

### 3- Loading the first data set

We add our first data set using the pandas library. Then we display the first 5 elements of our dataset using the panda's head function.

In [3]:
database1 = pd.read_csv("data.csv", sep=';')
database1.head()

,MUSK,2,3,4,5,6,7,8,9,10,...,159,160,161,162,163,164,165,166,167,168
0,MUSK,46,-108,-60,-69,-117,49,38,-161,-8,...,-308,52,-7,39,126,156,-50,-112,96,1.0
1,MUSK,41,-188,-145,22,-117,-6,57,-171,-39,...,-59,-2,52,103,136,169,-61,-136,79,1.0
2,MUSK,46,-194,-145,28,-117,73,57,-168,-39,...,-134,-154,57,143,142,165,-67,-145,39,1.0
3,MUSK,41,-188,-145,22,-117,-7,57,-170,-39,...,-60,-4,52,104,136,168,-60,-135,80,1.0
4,MUSK,41,-188,-145,22,-117,-7,57,-170,-39,...,-60,-4,52,104,137,168,-60,-135,80,1.0


### 4- Feature selection and splitting the train and test data sets

Let us denote the features we are going to use with X and response variable with y. With the drop function, we get the columns other than the columns 'musk' and '168'. For the response variable, we just take the column named '168'. Then we determine the size of our training and test sets. For this dataset, 90% of all data is used for training and the remaining 10% is used for testing using the function "train_test_split".

In [7]:
X = database1.drop(["MUSK","168"],axis=1)
y = database1['168']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.10 )

### 5. A generic function for training and evaluating a model

At this stage, we create a function that will run our 6 algorithms for our first dataset and give us 3 outputs. 

If we go into detail: first, we train our model with the features `X`, which is the training group we mentioned in Section 4, with the features in `y` as dependent variable. Then we make a prediction with the predict function according to the test data set. We record this prediction as `y_pred`. The test data set `y_test` is the test set of the results will give us the accuracy of the model. Then we print the classification report and confusion matrix with the help of the scikit-learn library. The explanation of all these functions will be found in the theory section of our thesis.

In [9]:
def model(name, X_train, y_train, X_test, y_test):
    name.fit(X_train, y_train)
    y_pred = name.predict(X_test)
    score= accuracy_score(y_test, y_pred) * 100
    print(str(name) + "Accuracy:",score)
    report_name=classification_report(y_test, y_pred)
    print(report_name)
    print(confusion_matrix(y_test,y_pred))

### 6. Recursive Feature Elimination

The purpose of the function below is to find the most suitable features for us in our database. The rfe (Recursive Feature Elimination) function produces an output based on the model we want and the number of desired features. It shows them in order, according to their effect.

In [10]:
def rfe(model,X,y):  
    rfe = RFE(model, 16)
    fit = rfe.fit(X, y)
    print("Num Features: %s" % (fit.n_features_))
    print("Selected Features: %s" % (fit.support_))
    print("Feature Ranking: %s" % (fit.ranking_))

### 7. Cross validation

Cross validation calculates the average accuracy and standard deviation for each part by dividing our data into as many parts as we want. This gives us an important idea of how well the model works.

In [11]:
def crossval(model,X,y):
    scores=cross_val_score(model, X, y, cv=10, scoring ="accuracy")
    print(str(model) + "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## 8. Experiments on the first data set

### 8.1 Decision Tree

Let us evaluate the Decision Tree model on our first data set.

In [15]:
model(dt,X_train,y_train,X_test,y_test)
crossval(dt,X,y)
rfe(dt,X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')Accuracy: 96.21212121212122
              precision    recall  f1-score   support

         0.0       0.97      0.98      0.98       549
         1.0       0.90      0.87      0.89       111

    accuracy                           0.96       660
   macro avg       0.94      0.93      0.93       660
weighted avg       0.96      0.96      0.96       660

[[538  11]
 [ 14  97]]
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrea

## 8.2 Random Forest

Let us evaluate the Random Forest algorithm on our first data set.

In [17]:
model(rf,X_train,y_train,X_test,y_test)
crossval(rf,X,y)
rfe(rf,X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)Accuracy: 97.12121212121212
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98       549
         1.0       0.99      0.84      0.91       111

    accuracy                           0.97       660
   macro avg       0.98      0.92      0.95       660
weighted avg       0.97      0.97      0.97       660

[[548   1]
 [ 18  93]]
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
        

## 8.3 Logistic Regression

Buradaki metni de yukaridaki gibi duzelt. Asagidakileri de...

In [18]:
model(logreg,X_train,y_train,X_test,y_test)
crossval(logreg,X,y)
rfe(logreg,X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)Accuracy: 95.0
              precision    recall  f1-score   support

         0.0       0.95      0.99      0.97       549
         1.0       0.92      0.77      0.84       111

    accuracy                           0.95       660
   macro avg       0.94      0.88      0.90       660
weighted avg       0.95      0.95      0.95       660

[[542   7]
 [ 26  85]]
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
               

## 8.4 Support Vector Machines (SVM)

Let's run the above 3 functions for the support vector machines(svm) algorithm.

In [19]:
model(svm,X_train,y_train,X_test,y_test)
crossval(svm,X,y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)Accuracy: 88.63636363636364
              precision    recall  f1-score   support

         0.0       0.88      1.00      0.94       549
         1.0       1.00      0.32      0.49       111

    accuracy                           0.89       660
   macro avg       0.94      0.66      0.71       660
weighted avg       0.90      0.89      0.86       660

[[549   0]
 [ 75  36]]
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)Accuracy: 0.85 (+/- 0.01)


## XFBoost (xgb)

Let's run the above 3 functions for the xgboost(xgb) algorithm.

In [20]:
model(xgb,X_train,y_train,X_test,y_test)
crossval(xgb,X,y)
rfe(xgb,X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)Accuracy: 99.0909090909091
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       549
         1.0       0.99      0.95      0.97       111

    accuracy                           0.99       660
   macro avg       0.99      0.98      0.98       660
weighted avg       0.99      0.99      0.99       660

[[548   1]
 [  5 106]]

## Gaussian Naive Bayes

Let's run the above 3 functions for the gaussian naive bayes(gnb) algorithm.

In [13]:
model(gnb,X_train,y_train,X_test,y_test)
crossval(gnb,X,y)


GaussianNB(priors=None, var_smoothing=1e-09)Accuracy: 83.03030303030303
              precision    recall  f1-score   support

         0.0       0.95      0.84      0.89       560
         1.0       0.46      0.76      0.58       100

    accuracy                           0.83       660
   macro avg       0.71      0.80      0.73       660
weighted avg       0.88      0.83      0.85       660

[[472  88]
 [ 24  76]]
GaussianNB(priors=None, var_smoothing=1e-09)Accuracy: 0.80 (+/- 0.31)


## 9. Our Second Data Set

Our second dataset is slightly different from the first one. The training and test sets consist of two different csv blocks, so we define these two datasets separately.

In [4]:
database2 = pd.read_csv("shuttle-train.csv", sep=';')
database2.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,d
0,50,21,77,0,28,0,27,48,22,2
1,55,0,92,0,0,26,36,92,56,4
2,53,0,82,0,52,-5,29,30,2,1
3,37,0,76,0,28,18,40,48,8,1
4,37,0,79,0,34,-26,43,46,2,1


In [5]:
database3 = pd.read_csv("shuttle-test.csv", sep=';')
database3.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,d
0,55,0,81,0,-6,11,25,88,64,4
1,56,0,96,0,52,-4,40,44,4,4
2,50,-1,89,-7,50,0,39,40,2,1
3,53,9,79,0,42,-2,25,37,12,4
4,55,2,82,0,54,-6,26,28,2,1


### 9.1 Train and Test Split

At this stage, we already have separate test and train subsets, and as a result, we do not need to use the `train_test_split` function we uses for the first dataset. Because currently our training and test sets are already given. All we have to do is to assign the independent and dependent variables.

In [6]:
X1_train = database2.drop(["d"], axis=1)
X1_test = database3.drop(["d"], axis=1)
y1_train = database2['d']
y1_test = database3['d']

### 9.2 Combining Train and Test for Cross-Validation

This stage is actually combining the operations we did in stage 9, the two datasets above and then assigning the properties as df_row_reindex_X and the results as df_row_reindex_y. As a result, it allows us to work with a single dataset, not two. We do this with the function named concat in the pandas library.

In [7]:
df_row_reindex = pd.concat([database2, database3], ignore_index=True)
df_row_reindex_X = df_row_reindex.drop(["d"], axis=1)
df_row_reindex_y = df_row_reindex['d']

## 9.3 Decision Trees

Let's run the above 3 functions for the decision tree(dt) algorithm.

In [13]:
model(dt,X1_train,y1_train,X1_test,y1_test)
crossval(dt,df_row_reindex_X,df_row_reindex_y)
rfe(dt,df_row_reindex_X,df_row_reindex_y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')Accuracy: 99.99310344827586
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     11478
           2       1.00      0.92      0.96        13
           3       1.00      1.00      1.00        39
           4       1.00      1.00      1.00      2155
           5       1.00      1.00      1.00       809
           6       1.00      1.00      1.00         4
           7       1.00      1.00      1.00         2

    accuracy                           1.00     14500
   macro avg       1.00      0.99      0.99     14500
weig

## 9.4 Random Forest

(rf) algorithm.

In [14]:
model(rf,X1_train,y1_train,X1_test,y1_test)
crossval(rf,df_row_reindex_X,df_row_reindex_y)
rfe(rf,df_row_reindex_X,df_row_reindex_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)Accuracy: 99.97931034482758
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     11478
           2       1.00      0.92      0.96        13
           3       0.97      1.00      0.99        39
           4       1.00      1.00      1.00      2155
           5       1.00      1.00      1.00       809
           6       1.00      0.75      0.86         4
           7       1.00      0.50      0.67         

## Let's run the above 3 functions for the logistic regression(logreg) algorithm.

In [15]:
model(logreg, X1_train,y1_train,X1_test,y1_test)
crossval(logreg,df_row_reindex_X,df_row_reindex_y)
rfe(logreg,df_row_reindex_X,df_row_reindex_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)Accuracy: 93.10344827586206
              precision    recall  f1-score   support

           1       0.93      0.99      0.96     11478
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        39
           4       0.91      0.61      0.73      2155
           5       1.00      1.00      1.00       809
           6       0.00      0.00      0.00         4
           7       0.00      0.00      0.00         2

    accuracy                           0.93     14500
   macro avg       0.41      0.37      0.38     14500
weighted avg       0.93      0.93      0.92     14500

[[11372     0     0   104     0     0     2]
 [

/home/kaygun/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kaygun/.local/lib/python3.8/site-packages/sklearn/svm/_base.py:946: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)Accuracy: 0.93 (+/- 0.01)
Num Features: 9
Selected Features: [ True  True  True  True  True  True  True  True  True]
Feature Ranking: [1 1 1 1 1 1 1 1 1]


## Let's run the above 3 functions for the xgboost(xgb) algorithm.

In [16]:
model(xgb,X1_train,y1_train,X1_test,y1_test)
crossval(xgb,df_row_reindex_X,df_row_reindex_y)
rfe(xgb,df_row_reindex_X,df_row_reindex_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)Accuracy: 99.99310344827586
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     11478
           2       1.00      0.92      0.96        13
           3       1.00      1.00      1.00        39
           4       1.00      1.00      1.00      2155
           5       1.00      1.00      1.00       809
           6       1.

## Let's run the above 3 functions for the gaussian naive bayes(gnb) algorithm.

In [17]:
model(gnb,X1_train,y1_train,X1_test,y1_test)
crossval(gnb,df_row_reindex_X,df_row_reindex_y)

GaussianNB(priors=None, var_smoothing=1e-09)Accuracy: 82.6551724137931
              precision    recall  f1-score   support

           1       0.95      0.88      0.92     11478
           2       0.01      0.92      0.02        13
           3       0.11      0.59      0.19        39
           4       0.89      0.54      0.67      2155
           5       0.99      0.82      0.90       809
           6       0.40      1.00      0.57         4
           7       0.00      1.00      0.01         2

    accuracy                           0.83     14500
   macro avg       0.48      0.82      0.47     14500
weighted avg       0.94      0.83      0.88     14500

[[10116   463   185   143     4     5   562]
 [    1    12     0     0     0     0     0]
 [    7     1    23     0     1     0     7]
 [  502   491     0  1162     0     0     0]
 [    0   142     0     0   666     1     0]
 [    0     0     0     0     0     4     0]
 [    0     0     0     0     0     0     2]]
GaussianNB(prior

### 14- We create two arrays for our third dataset. We will use these arrays for the functions we will write below. The difference between the two arrays is that the rfe function does not work in SVM and GNB algorithms.

In [19]:
models = [dt, rf, logreg, svm, xgb, gnb]

### 15- We throw our third dataset into the variable db.

In [20]:
db = pd.read_csv("data-hastalik.csv",sep = ';' )
db.head()

,ATES,BULANTI,BEL-AGRI,SUREKLI-WC,IDRAR-SIRASINDA-AGRI,URETRADA-YANMA-SISME-KASINTI,MESANE-ILTIHABI,BOBREK-ILTIHABI
0,355,0,1,0,0,0,0,0
1,359,0,0,1,1,1,1,0
2,359,0,1,0,0,0,0,0
3,360,0,0,1,1,1,1,0
4,360,0,1,0,0,0,0,0


### 16- At this stage, unlike the other two databases, we are not going to try our algorithms for two results, not one. Therefore, using the train_test_split function twice, we divide our data as a training and test, depending on the feature and result.

In [50]:
X = db.drop(["MESANE-ILTIHABI","BOBREK-ILTIHABI"],axis=1)
y = db['MESANE-ILTIHABI']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2 )

In [51]:
A = db.drop(["MESANE-ILTIHABI","BOBREK-ILTIHABI"],axis=1)
b = db['BOBREK-ILTIHABI']
A_train, A_test, b_train, b_test = train_test_split(A, b,test_size=0.2 )

### 17- We write 3 functions that we wrote for our two sets in one cell.

In [84]:
def model(name, f_train, f_test, d_train, d_test):
    name.fit(f_train, d_train)
    y_pred = name.predict(f_test)
    score= accuracy_score(d_test, y_pred) * 100
    print(str(name) + "Accuracy:",score)
    report_name=classification_report(d_test, y_pred)
    print(report_name)
    print(confusion_matrix(d_test,y_pred))
    
def rfe(model, features, decision):  
    rfe = RFE(model, 2)
    fit = rfe.fit(features, decision)
    print("Num Features: %s" % (fit.n_features_))
    print("Selected Features: %s" % (fit.support_))
    print("Feature Ranking: %s" % (fit.ranking_))
    
def crossval(model, features, decision , n):
    scores=cross_val_score(model, features, decision, cv=n , scoring ="accuracy")
    print(str(model) + "Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

### 18- For each algorithm and each output, we train the model and then implement cross_val.

In [85]:
for i in models:
    model(i, X_train, X_test, y_train, y_test)
    model(i, A_train, A_test, b_train, b_test)
    crossval(i, X, y, 5)
    crossval(i, X, y, 10)
    crossval(i, A, b, 5)
    crossval(i, A, b, 10)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')Accuracy: 100.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       1.00      1.00      1.00        10

    accuracy                           1.00        24
   macro avg       1.00      1.00      1.00        24
weighted avg       1.00      1.00      1.00        24

[[14  0]
 [ 0 10]]
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impu

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)Accuracy: 1.00 (+/- 0.00)
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)Accuracy: 1.00 (+/- 0.00)
SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)Accuracy: 87.5
              precision    re

### 19- Then we apply rfe in algorithms suitable for rfe function.

In [67]:
model = [dt, rf, logreg, xgb]
for i in model:
    print(i)
    rfe(i, X, y)
    rfe(i, A, b)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
Num Features: 2
Selected Features: [False False False  True  True False]
Feature Ranking: [2 5 4 1 1 3]
Num Features: 2
Selected Features: [ True False  True False False False]
Feature Ranking: [1 5 1 4 3 2]
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
             